
<h1 style="font-size: 3em; line-height:2em">Explainability and Bias Evaluation<br> with Tensorflow</h1> 

<br>
<br>
<br>
<br>
<br>
<br>

<h1> Alejandro Saucedo</h1>
Chief Scientist, The Institute for Ethical AI & Machine Learning

<br>
<br>

[github.com/ethicalml/bias-analysis](github.com/ethicalml/bias-analysis)


In [ ]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error, roc_curve, auc
import seaborn as sn
import matplotlib.pyplot as plt
import pandas as pd
import shap
from keras.layers import Input, Dense, Flatten, \
    Concatenate, concatenate, Dropout, Lambda
from keras.models import Model, Sequential
from keras.layers.embeddings import Embedding
import keras
from livelossplot import PlotLossesKeras
import eli5
from eli5.sklearn import PermutationImportance
import scipy
from scipy.cluster import hierarchy as hc
from pdpbox import pdp, get_dataset, info_plots
import math

params = {"ytick.color" : "w",
          "xtick.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w"}
plt.rcParams.update(params)


shap.initjs()


In [ ]:
label_column = "loan"
csv_path = 'data/adult.data'
csv_columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
                   "occupation", "relationship", "ethnicity", "gender", "capital-gain", "capital-loss",
                   "hours-per-week", "native-country", "loan"]
input_columns = ["age", "workclass", "education", "education-num", "marital-status",
                   "occupation", "relationship", "ethnicity", "gender", "capital-gain", "capital-loss",
                   "hours-per-week", "native-country"]
categorical_features = ["workclass", "education", "marital-status",
                       "occupation", "relationship", "ethnicity", "gender",
                       "native-country"]

In [ ]:

def prepare_data(df):
    
    if "fnlwgt" in df: del df["fnlwgt"]
    
    tmp_df = df.copy()

    # normalize data (this is important for model convergence)
    dtypes = list(zip(tmp_df.dtypes.index, map(str, tmp_df.dtypes)))
    for k,dtype in dtypes:
        if dtype == "int64":
            tmp_df[k] = tmp_df[k].astype(np.float32)
            tmp_df[k] -= tmp_df[k].mean()
            tmp_df[k] /= tmp_df[k].std()

    cat_columns = tmp_df.select_dtypes(['object']).columns
    tmp_df[cat_columns] = tmp_df[cat_columns].astype('category')
    tmp_df[cat_columns] = tmp_df[cat_columns].apply(lambda x: x.cat.codes)
    tmp_df[cat_columns] = tmp_df[cat_columns].astype('int8')
    
    return tmp_df

def get_dataset_1():
    tmp_df = df.copy()
    tmp_df = tmp_df.groupby('loan') \
                .apply(lambda x: x.sample(100) if not x["loan"].iloc[0] else x) \
                .reset_index(drop=True)
    
    X = tmp_df.drop(label_column, axis=1).copy()
    y = tmp_df[label_column].astype(int).values.copy()
    
    return tmp_df, df_display.copy()

def get_production_dataset():
    tmp_df = df.copy()
    
    X = tmp_df.drop(label_column, axis=1).copy()
    y = tmp_df[label_column].astype(int).values.copy()
    
    X_train, X_valid, y_train, y_valid = \
        train_test_split(X, y, test_size=0.2, random_state=7)

    return X_valid, y_valid


def get_dataset_2():
    tmp_df = df.copy()
    tmp_df_display = df_display.copy()
    
    X = tmp_df.drop(label_column, axis=1).copy()
    y = tmp_df[label_column].astype(int).values.copy()
    
    X_display = tmp_df_display.drop(label_column, axis=1).copy()
    y_display = tmp_df_display[label_column].astype(int).values.copy()
    
    X_train, X_valid, y_train, y_valid = \
        train_test_split(X, y, test_size=0.2, random_state=7)

    return X, y, X_train, X_valid, y_train, y_valid, X_display, y_display
    
df_display = pd.read_csv(csv_path, names=csv_columns)
df_display[label_column] = df_display[label_column].apply(lambda x: ">50K" in x)
df = prepare_data(df_display)

<h1 style="font-size: 5em">New Project!</h1>

<br>
<br>
<br>

## Client wants to automate loan approval process

<br>

#### They have a manual process where a domain expert goes through applicants
They want to automate this as they get 1m requests per month

In [55]:
df_data, df_display = get_dataset_1()

df_display.head(5)

,age,workclass,education,education-num,marital-status,occupation,relationship,ethnicity,gender,capital-gain,capital-loss,hours-per-week,native-country,loan
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,False
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,False
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,False
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,False
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,False


In [56]:
X = df_data.drop(label_column, axis=1).copy()
y = df_data[label_column].astype(int).values.copy()

X_train, X_valid, y_train, y_valid = \
        train_test_split(X, y, test_size=0.2, random_state=7)

X_train.head(5)

,age,workclass,education,education-num,marital-status,occupation,relationship,ethnicity,gender,capital-gain,capital-loss,hours-per-week,native-country
3337,-0.629133,4,9,1.134721,0,12,1,4,1,-0.145918,3.664620,-0.035429,39
4095,0.470539,4,9,1.134721,2,3,0,1,1,-0.145918,-0.216656,-0.035429,24
7699,1.203653,6,14,1.912109,6,10,1,4,1,-0.145918,-0.216656,1.179399,39
3773,0.397227,4,11,-0.420053,2,1,0,4,1,-0.145918,-0.216656,0.612479,39
1592,0.103982,4,9,1.134721,2,12,0,4,1,-0.145918,-0.216656,-0.035429,39


In [ ]:
def build_model():
    input_els = []
    encoded_els = []
    dtypes = list(zip(X.dtypes.index, map(str, X.dtypes)))
    for k,dtype in dtypes:
        input_els.append(Input(shape=(1,)))
        if dtype == "int8":
            e = Flatten()(Embedding(df[k].max()+1, 1)(input_els[-1]))
        else:
            e = input_els[-1]
        encoded_els.append(e)
    encoded_els = concatenate(encoded_els)

    layer1 = Dropout(0.5)(Dense(100, activation="relu")(encoded_els))
    out = Dense(1, activation='sigmoid')(layer1)

    # train model
    model = Model(inputs=input_els, outputs=[out])
    model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])
    return model

def f_in(X, m=None):
    """Preprocess input so it can be provided to a function"""
    if m:
        return [X.iloc[:m,i] for i in range(X.shape[1])]
    else:
        return [X.iloc[:,i] for i in range(X.shape[1])]

def f_out(probs):
    """Convert probabilities into classes"""
    return list((probs >= 0.5).astype(int).T[0])

In [ ]:
model = build_model()

model.fit(
    f_in(X_train), y_train, epochs=50,
    batch_size=512, shuffle=True, validation_data=(f_in(X_valid), y_valid),
    callbacks=[PlotLossesKeras()], verbose=0, validation_split=0.05,
)

In [ ]:
score = model.evaluate(f_in(X_valid), y_valid, verbose=1)
print("Error %.4f: " % score[0])
print("Accuracy %.4f: " % (score[1]*100))

Accuracy is 98%

# Time for PROD!


2am call....

...what do you mean performing terrible?

# Time to REVERT!

<img src="files/subdir/image.png">

In [ ]:
X_prod, y_prod = get_production_dataset()

score = model.evaluate(f_in(X_prod), y_prod, verbose=1)

print("Error %.4f: " % score[0])
print("Accuracy %.4f: " % (score[1]*100))

In [ ]:
probabilities = model.predict(f_in(X_valid))
print(list(probabilities.T[0])[:10])

pred = f_out(probabilities)
print(pred[:10])

In [ ]:
confusion = sklearn.metrics.confusion_matrix(y_valid, pred)
confusion_scaled = confusion.astype("float") / confusion.sum(axis=1)[:, np.newaxis]
confusion_scaled_df = pd.DataFrame(confusion_scaled, index=["Denied", "Approved"], columns=["Denied", "Approved"])

sn.heatmap(confusion_scaled_df, annot=True, fmt='.2f', center=1)
plt.show()

In [ ]:
confusion_df = pd.DataFrame(confusion,
            index=["Actual Denied", "Actual Approved"], 
            columns=["Preditced Denied", "Preditced Approved"])

sn.heatmap(confusion_df, annot=True, fmt='d', center=1)
plt.show()

In [ ]:
def plot_roc(y, probs):
    
    fpr, tpr, _ = roc_curve(y, probs)

    roc_auc = auc(fpr, tpr)
    print(roc_auc)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="lower right")
    plt.rcParams.update(params)
    plt.show()


In [ ]:
print(sklearn.metrics.classification_report(y_valid, pred))

plot_roc(y_valid, pred)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,7))
sn.countplot(y_valid, ax=ax[0]) 
sn.countplot(y_prod, ax=ax[1]) 

In [ ]:
X, y, X_train, X_valid, y_train, y_valid, X_display, y_display \
    = get_dataset_2()
X.head()

# Learning curves

In [ ]:
def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    train_errors, val_errors = [], []
    for m in list(np.logspace(0.6,4,dtype='int')):
        if m >= len(X_train): break
        model.fit(f_in(X_train,m), y_train[:m], epochs=50, batch_size=512, verbose=0)
        y_train_predict = model.predict(f_in(X_train,m))
        y_val_predict = model.predict(f_in(X_val))
        y_train_predict = f_out(y_train_predict)
        y_val_predict = f_out(y_val_predict)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))
    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")


In [ ]:
plot_learning_curves(model, X_valid, y_valid)

# SHAP for TF/Keras

In [ ]:
def shap_predict(X):
    values = model.predict([X[:,i] for i in range(X.shape[1])]).flatten()
    return values

shap_explainer = shap.KernelExplainer(shap_predict, X.iloc[:100,:])

In [ ]:
shap_idx = 0
shap_x = X.iloc[shap_idx,:]
shap_display_x = X_display.iloc[shap_idx,:]
shap_values = shap_explainer.shap_values(shap_x, nsamples=500)

In [ ]:

print("Probability: ", probabilities[shap_idx])
shap.force_plot(shap_explainer.expected_value, shap_values, shap_display_x, matplotlib=True, figsize=(40, 3))


In [ ]:
shap_idx_multi_from = 15
shap_idx_multi_to = 40

In [ ]:
shap_x_multi = X.iloc[shap_idx_multi_from:shap_idx_multi_to,:]
shap_display_multi = X_display.iloc[shap_idx_multi_from:shap_idx_multi_to,:]
shap_y_multi = y[shap_idx_multi_from:shap_idx_multi_to]

In [ ]:
shap_values_multi = shap_explainer.shap_values(shap_x_multi, nsamples=500)

In [ ]:
shap.force_plot(shap_explainer.expected_value, shap_values_multi, shap_display_multi)

In [ ]:
shap.dependence_plot("hours-per-week", 
                     shap_values_multi, 
                     shap_x_multi, 
                     display_features=shap_display_multi,
                     axis_color='#FFFFFF')


In [ ]:
shap.summary_plot(shap_values_multi, shap_x_multi, axis_color='#FFFFFF')


# LIME on Keras+Tensorflow

In [ ]:
from lime.lime_tabular import LimeTabularExplainer

In [ ]:
tf_lime_explainer = LimeTabularExplainer(
                X_train.values,
                feature_names=list(X_train.columns),
                categorical_features=categorical_features)

In [ ]:
def lime_predict_proba(X):
    values = model.predict([X[:,i] for i in range(X.shape[1])]).flatten()
    prob_pairs = np.array([1-values, values]).T
    return prob_pairs

tf_lime_explanation = tf_lime_explainer.explain_instance(
        X_train.iloc[1,:], lime_predict_proba, num_features=13) 

In [ ]:
tf_lime_explanation.as_pyplot_figure()

In [ ]:
tf_lime_explanation.show_in_notebook(show_table=True, show_all=True)


# Data Analysis

In [ ]:
X_analysis = X_display.copy()
y_analysis = y_display.copy()
df_analysis = X_analysis.copy()
df_analysis[label_column] = y_analysis

In [ ]:
df[label_column].value_counts().plot.barh()

In [ ]:
fig = plt.figure(figsize=(20,15))
cols = 5
rows = math.ceil(float(X.shape[1]) / cols)
for i, column in enumerate(X.columns):
    ax = fig.add_subplot(rows, cols, i + 1)
    ax.set_title(column)
    if X.dtypes[column] == np.object:
        X[column].value_counts().plot(kind="bar", axes=ax)
    else:
        X[column].hist(axes=ax)
        plt.xticks(rotation="vertical")
plt.subplots_adjust(hspace=0.7, wspace=0.2)

In [ ]:
df["gender"].value_counts().plot.barh()

In [ ]:
flds = ["gender", "age", "education-num", "capital-gain", 
                  "capital-loss", "hours-per-week"]


enc_summ = X_display[flds].groupby("gender", as_index=False).mean()
enc_summ

In [ ]:
df["ethnicity"].value_counts().plot.barh()

In [ ]:
def keras_score(self, X, y, **kwargs):
    input_test = [X[:,i] for i in range(X.shape[1])]
    loss = self.evaluate(input_test, y)
    if type(loss) is list:
        # The first one is the error, the rest are metrics
        return -loss[0]
    return -loss

In [ ]:
perm = PermutationImportance(model, random_state=1, scoring=keras_score).fit(X,y)

In [ ]:
eli5_fi = eli5.explain_weights_df(perm, feature_names = X.columns.tolist())
eli5_fi

In [ ]:
eli5_fi.plot("feature", "weight", "barh", figsize=(12,7))

In [ ]:
df.groupby(["marital-status", "loan"]).size().plot("barh")

# Sit down with our clients to understand these perspectives

In [ ]:
def plot_dendogram(corr):
    corr_condensed = hc.distance.squareform(1-corr)
    z = hc.linkage(corr_condensed, method="average")
    fig = plt.figure(figsize=(16,5))
    dendrogram = hc.dendrogram(
        z, labels=X_analysis.columns, orientation="left", leaf_font_size=16)
    plt.show()

In [ ]:
corr = np.round(scipy.stats.spearmanr(X_analysis).correlation, 4)
plot_dendogram(corr)

# Partial dependence

In [ ]:
df_analysis.plot(
    "age", 
    "capital-gain",
    "scatter",
    alpha=0.01,
    figsize=(10,8))

In [ ]:
fig, axes, summary_df = info_plots.target_plot(
    df=df, feature="age", feature_name="age", target="capital-gain")

In [ ]:
class ModelWrapper():
    def __init__(self, model):
        self.model = model
    
    def predict(self, X, **kwargs):
        return self.model.predict([X.iloc[:,i] for i in range(X.shape[1])])

In [ ]:
mr = ModelWrapper(model)
    
fig, axes, summary_df = info_plots.actual_plot(
    model=mr, X=X, feature="gender", feature_name="gender", predict_kwds={})

In [ ]:
pdp_sex = pdp.pdp_isolate(
    model=mr, dataset=X, model_features=input_columns, feature='gender'
)

fig, axes = pdp.pdp_plot(pdp_sex, 'gender')